# **Dessert Flavor Combinations**
The goal here is predict what flavor combinations make for a good dessert. E.g. history tells us apple, cinammon, and nutmeg pair well because a large number of recipies use these flavors and these recipies are generally well rated. Here, I attempt to do this with a data set scrapped from epicurious.com. Basically, the features are a list of flavors present in the recipie and the target is the rating (0 to 4 stars)

This notebook is an attempt to apply a NN instead of a SVR, WORK IN PROGRESS!!!!!

In [1]:
%matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk as nltk
import string

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

/home/lgriffin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1) Load data

In [2]:
dataset = pd.read_csv('recipes.csv')

## 2) Clean Data

In [3]:
#Remove entires that dont have strings for the flavors
dataset = dataset[dataset['flavors'].apply(lambda x: isinstance(x, str))]

#tokenize the flavor string 
temp = dataset['flavors'].apply(lambda x: np.unique(nltk.word_tokenize(x)))

#Calc number of flavors
dataset.loc[:,'num_flavors'] = temp.apply(lambda x: x.shape[0])

#Remove entries with only 1 flavor
dataset = dataset[dataset['num_flavors'] > 1]

#convert rating to float
dataset.loc[:,'rating'] = dataset['rating'].apply(lambda x: float(x))

dataset.loc[:,'flavors_clean'] = temp.apply(lambda x: "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in x]).strip())

#So, one thing I have yet to do is account for recipes that have no reviews...
#I need to re-scrape the data set to get that info, for now lets just remove
#recipies with a rating of 0
dataset = dataset[dataset['rating'] != 0]

## 3) Preprocess

In [4]:
#Get features and targets
features = dataset['flavors_clean'].values
targets = dataset['rating'].values

In [13]:
#Set up a pipeline:
#   1) Use TF-IDF to generate features
#   2) maybe other things later????
vectorizer = make_pipeline(
    TfidfVectorizer(sublinear_tf=True)
)

x_train = vectorizer.fit_transform(features).todense()

## 4) Model

In [20]:
model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.softmax),
    keras.layers.Dense(1)
])

In [21]:
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001), 
              loss='mse',
              metrics=['mae'])

## 6) Train

In [26]:
epocs = 10
dessert_rating = model.fit(x_train, targets, epochs=epocs, validation_split=0.2)

Train on 4632 samples, validate on 1159 samples
Epoch 1/10
4632/4632 [==============================] - 0s 98us/step - loss: 0.3189 - mean_absolute_error: 0.4358 - val_loss: 0.2365 - val_mean_absolute_error: 0.3755
Epoch 2/10
4632/4632 [==============================] - 1s 137us/step - loss: 0.3189 - mean_absolute_error: 0.4345 - val_loss: 0.2361 - val_mean_absolute_error: 0.3770
Epoch 3/10
4632/4632 [==============================] - 1s 152us/step - loss: 0.3190 - mean_absolute_error: 0.4362 - val_loss: 0.2363 - val_mean_absolute_error: 0.3762
Epoch 4/10
4632/4632 [==============================] - 1s 131us/step - loss: 0.3190 - mean_absolute_error: 0.4357 - val_loss: 0.2363 - val_mean_absolute_error: 0.3763
Epoch 5/10
4632/4632 [==============================] - 1s 137us/step - loss: 0.3188 - mean_absolute_error: 0.4364 - val_loss: 0.2375 - val_mean_absolute_error: 0.3733
Epoch 6/10
4632/4632 [==============================] - 1s 138us/step - loss: 0.3190 - mean_absolute_error: 0.434

Not better than the SVR!!! so far..